In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Download dataset
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=dataset_url, extract=True)
data_dir = path_to_zip.replace(".zip", "")
train_dir = data_dir + "/cats_and_dogs_filtered/train"
val_dir = data_dir + "/cats_and_dogs_filtered/validation"

# Data preprocessing + augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
val_data = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Build Model (Transfer Learning)
base_model = MobileNetV2(input_shape=(224,224,3), weights="imagenet", include_top=False)
for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=3, validation_data=val_data)

# Print accuracy
print("✅ Training Accuracy:", history.history['accuracy'][-1])
print("✅ Validation Accuracy:", history.history['val_accuracy'][-1])


68606236/68606236 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.8800 - loss: 0.2476 - val_accuracy: 0.9800 - val_loss: 0.0594
Epoch 2/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.9716 - loss: 0.0793 - val_accuracy: 0.9780 - val_loss: 0.0636
Epoch 3/3
63/63 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9710 - loss: 0.0749 - val_accuracy: 0.9720 - val_loss: 0.0771
✅ Training Accuracy: 0.968500018119812
✅ Validation Accuracy: 0.972000002861023
